In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import scanpy as sc
import anndata as ad
import numpy as np
import pandas as pd
import anndata2ri
import gdown
import scipy
import scipy.io
from rpy2.robjects import r

anndata2ri.activate()

In [2]:
%load_ext rpy2.ipython

In [3]:
%%R
suppressPackageStartupMessages({
    library(SingleCellExperiment)
    library(Seurat)
    library(SeuratDisk)
})

# Load data

In [104]:
adata = sc.read('../data/10x-cite/10x-cite.h5ad')
adata

AnnData object with n_obs × n_vars = 10849 × 15792
    obs: 'n_genes', 'percent_mito', 'n_counts', 'batch', 'labels'
    var: 'encode', 'n_cells-0', 'highly_variable-0', 'hvg_encode-0', 'n_cells-1', 'highly_variable-1', 'hvg_encode-1'
    obsm: 'protein_expression'

In [107]:
# Seurat categories have to be strings
adata.obs.batch = adata.obs.batch.astype('category')
adata.obs.batch = adata.obs.batch.cat.rename_categories(["0", "1"])

In [108]:
query = adata[adata.obs.batch == '1']
query

View of AnnData object with n_obs × n_vars = 3994 × 15792
    obs: 'n_genes', 'percent_mito', 'n_counts', 'batch', 'labels'
    var: 'encode', 'n_cells-0', 'highly_variable-0', 'hvg_encode-0', 'n_cells-1', 'highly_variable-1', 'hvg_encode-1'
    obsm: 'protein_expression'

In [7]:
adata = adata[adata.obs.batch == '0']
adata

View of AnnData object with n_obs × n_vars = 6855 × 4000
    obs: 'n_genes', 'percent_mito', 'n_counts', 'batch', 'labels'
    var: 'encode', 'n_cells-0', 'highly_variable-0', 'hvg_encode-0', 'n_cells-1', 'highly_variable-1', 'hvg_encode-1', 'highly_variable', 'highly_variable_rank', 'means', 'variances', 'variances_norm'
    uns: 'hvg'
    obsm: 'protein_expression'

In [8]:
adata_ = ad.AnnData(adata.X)
adata_.obs_names = adata.obs_names
adata_.var_names = adata.var_names
adata_.obs['cell_type'] = adata.obs['labels']

In [9]:
%%R -i adata_
rna = as.Seurat(adata_, counts='X', data=NULL)
rna

An object of class Seurat 
4000 features across 6855 samples within 1 assay 
Active assay: RNA (4000 features, 0 variable features)


In [10]:
adata_ = ad.AnnData(adata.obsm['protein_expression'])
adata_.obs['cell_type'] = adata.obs['labels']

In [11]:
%%R -i adata_
cite = as.Seurat(adata_, counts='X', data=NULL)
cite

R[write to console]: Warning:
R[write to console]:  Feature names cannot have underscores ('_'), replacing with dashes ('-')



An object of class Seurat 
14 features across 6855 samples within 1 assay 
Active assay: RNA (14 features, 0 variable features)


In [12]:
%%R
bm <- rna
bm[["ADT"]] <- CreateAssayObject(counts = cite@assays$RNA@counts)

rm(rna)
rm(cite)

print('Preprocessing RNA...')
DefaultAssay(bm) <- "RNA"
bm <- SCTransform(bm, verbose = FALSE)
VariableFeatures(bm) <- rownames(bm[["RNA"]])
#bm <- NormalizeData(bm, verbose=FALSE)
#VariableFeatures(bm) <- rownames(bm[["RNA"]])
bm <- ScaleData(bm, verbose=FALSE)

bm <- RunPCA(bm, verbose=FALSE)

print('Preprocessing ADT...')
DefaultAssay(bm) <- "ADT"
VariableFeatures(bm) <- rownames(bm[["ADT"]])
bm <- NormalizeData(bm, normalization.method = 'CLR', margin = 2, verbose=FALSE)
bm <- ScaleData(bm, verbose=FALSE)
bm <- RunPCA(bm, reduction.name = "apca", verbose=FALSE)

[1] "Preprocessing RNA..."


R[write to console]: Warning:
R[write to console]:  Not all features provided are in this Assay object, removing the following feature(s): AL034417.3, FHAD1, AL603832.1, PDZK1, AL356441.1, TGFB2, MIR3681HG, LINC01412, CHL1, ZBTB20-AS4, LINC02029, AC092546.1, AC084871.2, MAST4-AS1, AL358933.1, ULBP1, GHET1, AFF2, L1CAM, AC107959.4, SCARA5, ARC, SMC2-AS1, TMEM236, FZD8, ADIRF, KCNA5, TRAV8-4, AL162311.3, AL139099.1, AC010536.1, SAMD14, KCNB1, PHACTR3, SIGLEC6, PCDH11Y, IGLV6-57, CRYBB1, AL035681.1, RSPH1



[1] "Preprocessing ADT..."


R[write to console]: Warning in irlba(A = t(x = object), nv = npcs, ...) :
R[write to console]: 
 
R[write to console]:  You're computing too large a percentage of total singular values, use a standard svd instead.



In [13]:
%%R
bm <- FindMultiModalNeighbors(
       bm, reduction.list = list("pca", "apca"), 
       dims.list = list(1:30, 1:13), modality.weight.name = "RNA.weight"
   )

#bm <- RunSPCA(bm, assay = 'RNA', graph = 'wsnn', npcs = 20)

#SaveH5Seurat(bm, 'hao-seurat.h5seurat', overwrite = TRUE)

R[write to console]: Calculating cell-specific modality weights

R[write to console]: Finding 20 nearest neighbors for each modality.



  |                                                  | 0 % ~calculating   |+++++++++++++++++++++++++                         | 50% ~04s           |++++++++++++++++++++++++++++++++++++++++++++++++++| 100% elapsed=09s  


R[write to console]: Calculating kernel bandwidths



  |                                                  | 0 % ~calculating   |+++++++++++++++++++++++++                         | 50% ~00s           |++++++++++++++++++++++++++++++++++++++++++++++++++| 100% elapsed=01s  


R[write to console]: Finding multimodal neighbors



  |                                                  | 0 % ~calculating   |+++++++++++++++++++++++++                         | 50% ~13s           |++++++++++++++++++++++++++++++++++++++++++++++++++| 100% elapsed=27s  
  |                                                  | 0 % ~calculating   |+++++++++++++++++++++++++                         | 50% ~01s           |++++++++++++++++++++++++++++++++++++++++++++++++++| 100% elapsed=02s  


R[write to console]: Constructing multimodal KNN graph

R[write to console]: Constructing multimodal SNN graph



In [14]:
adata_ = ad.AnnData(query.X)
adata_.obs_names = query.obs_names
adata_.var_names = query.var_names
adata_.obs['cell_type'] = query.obs['labels']

In [15]:
%%R -i adata_
query = as.Seurat(adata_, counts='X', data=NULL)
query

An object of class Seurat 
4000 features across 3994 samples within 1 assay 
Active assay: RNA (4000 features, 0 variable features)


In [16]:
%%R
query <- SCTransform(query, verbose = FALSE)

In [21]:
%%R
DefaultAssay(bm) <- "RNA"
VariableFeatures(bm) <- rownames(bm[["RNA"]])
bm

An object of class Seurat 
7974 features across 6855 samples within 3 assays 
Active assay: RNA (4000 features, 4000 variable features)
 2 other assays present: ADT, SCT
 2 dimensional reductions calculated: pca, apca


In [23]:
%%R
bm <- RunSPCA(bm, assay = 'SCT', graph = 'wsnn', npcs = 20)

R[write to console]: Computing sPCA transformation



In [26]:
%%R
DefaultAssay(bm) <- "RNA"
bm[['SCT']]

SCTAssay data with 3960 features for 6855 cells, and 1 SCTModel(s) 
Top 10 variable features:
 AL645608.8, HES4, ISG15, TTLL10, TNFRSF18, TNFRSF4, MXRA8, ANKRD65,
ATAD3C, AL645728.1 


In [31]:
%%R
anchors <- FindTransferAnchors(
  reference = bm,
  reference.assay = 'SCT',
  query = query,
  normalization.method = "SCT",
  reference.reduction = "pca",
  dims = 1:30
)

R[write to console]: Normalizing query using reference SCT model

R[write to console]: Projecting cell embeddings

R[write to console]: Finding neighborhoods

R[write to console]: Finding anchors

R[write to console]: 	Found 5803 anchors



In [32]:
%%R
query <- TransferData(
  anchorset = anchors, 
  reference = bm,
  query = query,
  refdata = list(
    predicted_ADT = "ADT")
)

R[write to console]: Finding integration vectors

R[write to console]: Finding integration vector weights

R[write to console]: 0%   10   20   30   40   50   60   70   80   90   100%

R[write to console]: [----|----|----|----|----|----|----|----|----|----|

R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[writ

In [38]:
%%R
DefaultAssay(query) <- "predicted_ADT"
query

An object of class Seurat 
7958 features across 3994 samples within 3 assays 
Active assay: predicted_ADT (14 features, 0 variable features)
 2 other assays present: RNA, SCT


In [39]:
%%R -o query_sc
query_sc <- as.SingleCellExperiment(query)
query_sc <- as(query_sc, 'SingleCellExperiment')

In [101]:
%%R
SaveH5Seurat(bm, 'ref-imput.h5seurat', overwrite = TRUE)
SaveH5Seurat(query, 'query-imput.h5seurat', overwrite = TRUE)

R[write to console]: Creating h5Seurat file for version 3.1.5.9900

R[write to console]: Adding counts for RNA

R[write to console]: Adding data for RNA

R[write to console]: Adding variable features for RNA

R[write to console]: No feature-level metadata found for RNA

R[write to console]: Adding counts for ADT

R[write to console]: Adding data for ADT

R[write to console]: Adding scale.data for ADT

R[write to console]: Adding variable features for ADT

R[write to console]: No feature-level metadata found for ADT

R[write to console]: Adding counts for SCT

R[write to console]: Adding data for SCT

R[write to console]: Adding scale.data for SCT

R[write to console]: Adding variable features for SCT

R[write to console]: No feature-level metadata found for SCT

R[write to console]: Writing out SCTModel.list for SCT

R[write to console]: Adding cell embeddings for pca

R[write to console]: Adding loadings for pca

R[write to console]: No projected loadings for pca

R[write to console]:

In [116]:
from scipy.stats import pearsonr

protein_corrs = []
for i, protein in enumerate(query_sc.var_names):
    protein = protein[:-10]
    protein_corrs.append(protein + ': Corr=' + str(np.round(pearsonr(np.log1p(query_sc.obsm['predicted_protein_expression'][:, i]), np.log1p(query.obsm['protein_expression'].iloc[:, i].values))[0], 3)))
    
protein_corrs

['CD3: Corr=0.94',
 'CD4: Corr=0.894',
 'CD8a: Corr=0.739',
 'CD14: Corr=0.956',
 'CD15: Corr=0.236',
 'CD16: Corr=0.785',
 'CD56: Corr=0.768',
 'CD19: Corr=0.887',
 'CD25: Corr=0.522',
 'CD45RA: Corr=0.804',
 'CD45RO: Corr=0.786',
 'PD-1: Corr=0.475',
 'TIGIT: Corr=0.598',
 'CD127: Corr=0.838']